In [253]:
import pymysql
import pandas as pd
import numpy as np

In [254]:
df = pd.read_csv('tabla_de_hechosP.csv')

In [255]:
df.rename(columns={'Remittance inflows (US$ million)': 'Remittance'}, inplace=True)

In [256]:
df.rename(columns={'GDP (current US$': 'GDP'}, inplace=True)

In [257]:
df.rename(columns={'Inflation, consumer prices (annual %)': 'Inflation'}, inplace=True)

In [258]:
df.rename(columns={'Net migration': 'Net_migration'}, inplace=True)

In [259]:
df.rename(columns={'aux': 'Id'}, inplace=True)

In [260]:
df.head()

,anio,pais,Remittance,Id,GDP,Inflation,Net_migration
0,2012,India,68820.517838,2012India,1.827638e+12,9.478997,-180991.0
1,2013,India,69970.360847,2013India,1.856721e+12,10.017878,-236387.0
2,2014,India,70388.642797,2014India,2.039126e+12,6.665657,-310694.0
3,2015,India,68909.693353,2015India,2.103588e+12,4.906973,-475094.0
4,2016,India,62744.364090,2016India,2.294797e+12,4.948216,68119.0


In [261]:
df['pais'] = df['pais'].astype(str)

In [262]:
df['Id'] = df['Id'].astype(str)

In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   anio           45 non-null     int64  
 1   pais           45 non-null     object 
 2   Remittance     45 non-null     float64
 3   Id             45 non-null     object 
 4   GDP            45 non-null     float64
 5   Inflation      45 non-null     float64
 6   Net_migration  45 non-null     float64
dtypes: float64(4), int64(1), object(2)
memory usage: 2.6+ KB


In [264]:
#Establezco conexion
conexion = pymysql.connect(
    host = "database-migraciones.c5vthlbc6wtm.us-east-1.rds.amazonaws.com",
    user= "***********",
    passwd = "***********",
    db = "migraciones"

)

In [265]:
cursor = conexion.cursor()

In [266]:
cursor.execute("""DROP TABLE IF EXISTS macroeconomico""")
cursor.execute("""DROP TABLE IF EXISTS anio""")
cursor.execute("""DROP TABLE IF EXISTS pais""")

0

In [267]:
cursor.execute("""CREATE TABLE macroeconomico (
    Id VARCHAR(45) PRIMARY KEY,
    anio INT,
    pais VARCHAR(45),
    remittance FLOAT,
    GDP FLOAT,
    Inflation FLOAT,
    Net_migration FLOAT
)""")

0

In [268]:
cursor.execute("""CREATE TABLE anio (
    id_anio INT AUTO_INCREMENT PRIMARY KEY,
    anio INT
)""")

0

In [269]:
cursor.execute("""CREATE TABLE pais (
    id_pais INT AUTO_INCREMENT PRIMARY KEY,
    pais VARCHAR(45)
)""")

0

In [270]:
df_lista = []
for i in range (0,45):
    df_lista.append((df.anio[i], df.pais[i],df.Remittance[i], df.Id[i],df.GDP[i],df.Inflation[i],df.Net_migration[i])) 

In [271]:
cursor.executemany("""insert into macroeconomico (anio, pais, remittance, Id,GDP, Inflation, Net_migration)
                    values (%s, %s, %s, %s, %s, %s, %s)""", df_lista
)

45

In [272]:
conexion.commit()

In [273]:
cursor.execute("""INSERT INTO anio(anio) SELECT DISTINCT anio FROM macroeconomico""")

9

In [274]:
conexion.commit()

In [275]:
cursor.execute("""INSERT INTO pais(pais) SELECT DISTINCT pais FROM macroeconomico""")

5

In [276]:
conexion.commit()

In [277]:
cursor.execute("""ALTER TABLE macroeconomico
                ADD id_anio INT""")

0

In [278]:
cursor.execute("""UPDATE macroeconomico
                SET id_anio = 0""")

45

In [279]:
cursor.execute("""UPDATE macroeconomico
                JOIN anio ON (macroeconomico.anio = anio.anio)
                SET macroeconomico.id_anio = anio.id_anio""")

45

In [280]:
cursor.execute("""ALTER TABLE macroeconomico
                DROP COLUMN anio""")

0

In [281]:
cursor.execute("""ALTER TABLE macroeconomico
                ADD id_pais INT""")

0

In [282]:
cursor.execute("""UPDATE macroeconomico
                SET id_pais = 0""")

45

In [283]:
cursor.execute("""UPDATE macroeconomico
                JOIN pais ON (macroeconomico.pais = pais.pais)
                SET macroeconomico.id_pais = pais.id_pais""")

45

In [284]:
cursor.execute("""ALTER TABLE macroeconomico
                DROP COLUMN pais""")

0

In [285]:
cursor.execute("""ALTER TABLE macroeconomico 
                    ADD CONSTRAINT macroeconomico_fk_anio FOREIGN KEY (id_anio) REFERENCES anio(id_anio)""")

45

In [286]:
cursor.execute("""ALTER TABLE macroeconomico 
                    ADD CONSTRAINT macroeconomico_fk_pais FOREIGN KEY (id_pais) REFERENCES pais(id_pais)""")


45

In [287]:
conexion.commit()

In [288]:
conexion.close()